In [ ]:
!pip install jsonlines

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
import tensorflow as tf
import nltk
import re 
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
import jsonlines
from tqdm.auto import tqdm
tqdm.pandas()

In [6]:
path = "/content/drive/MyDrive/jd_data_1000.jsonl"

text = []
label = []
title = []
i=0

with jsonlines.open(path) as f:
    for line in f.iter():
        text.append(line['text'])
        label.append(line['label'])
        title.append(line["title"])
        i+=1

df = pd.DataFrame({'text':text, "title":title, 'label':label})
df

,text,title,label
0,Search by Keyword More Options Search by Locat...,Loss Prevention Detective - Colma,Accounting
1,Description At more than 80 award-winning prop...,Loss Prevention Officer PM,Accounting
2,COMPANY DESCRIPTION: Ethos Risk Services is a ...,P/T Special Investigations Unit Field Investig...,Accounting
3,You’ve always been hyper alert to your surroun...,Loss Prevention/Hotel Security - On Call,Accounting
4,Your surprise is waiting! Explore career oppor...,Loss Prevention Detective - Millbury,Accounting
...,...,...,...
19995,"PetroChoice, a leading distributor of lubrican...",Warehouse Specialist 9a-6:30p,Warehousing
19996,Old Dominion is currently recruiting for a Doc...,Dockworker - Part Time (4 PM to 10 PM),Warehousing
19997,"At American Signature Inc., we believe everyon...",Warehouse Associate,Warehousing
19998,"Pennsylvania 700 Uline Way, Allentown, PA 1810...",Warehouse Associate - Nights - Up to $25 per hour,Warehousing


In [7]:
df['label'].value_counts()

Accounting                                                   1000
Administrative Support                                       1000
Transportation                                               1000
Trade - Retail                                               1000
Sales & Business Development                                 1000
Pharmaceutical/Biotechnology                                 1000
Marketing & Advertising                                      1000
Manufacturing                                                1000
Legal Services                                               1000
Information & Media - Publishing                             1000
Information & Media - Internet & Information Technologies    1000
Information & Media - Broadcasting                           1000
Human Resources                                              1000
Hospitality - Food Services                                  1000
Health Care & Caregiving                                     1000
Finance - 

In [8]:
label_encoder = LabelEncoder()
encoded_label = label_encoder.fit_transform(df['label'])
df['label'] = encoded_label
df

,text,title,label
0,Search by Keyword More Options Search by Locat...,Loss Prevention Detective - Colma,0
1,Description At more than 80 award-winning prop...,Loss Prevention Officer PM,0
2,COMPANY DESCRIPTION: Ethos Risk Services is a ...,P/T Special Investigations Unit Field Investig...,0
3,You’ve always been hyper alert to your surroun...,Loss Prevention/Hotel Security - On Call,0
4,Your surprise is waiting! Explore career oppor...,Loss Prevention Detective - Millbury,0
...,...,...,...
19995,"PetroChoice, a leading distributor of lubrican...",Warehouse Specialist 9a-6:30p,19
19996,Old Dominion is currently recruiting for a Doc...,Dockworker - Part Time (4 PM to 10 PM),19
19997,"At American Signature Inc., we believe everyon...",Warehouse Associate,19
19998,"Pennsylvania 700 Uline Way, Allentown, PA 1810...",Warehouse Associate - Nights - Up to $25 per hour,19


In [10]:
def clean_text(text):
    # convert to lowercase
    text = text.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return " ".join(words)

In [13]:
df['cleantext'] = df['text'].progress_apply(clean_text)
df['cleantitle'] = df['title'].progress_apply(clean_text)
df

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

,text,title,label,cleantext
0,Search by Keyword More Options Search by Locat...,loss prevent detect colma,0,search keyword option search locat clear load ...
1,Description At more than 80 award-winning prop...,loss prevent offic pm,0,descript 80 award win properti worldwid ritz c...
2,COMPANY DESCRIPTION: Ethos Risk Services is a ...,p special investig unit field investig siu,0,compani descript etho risk servic nation renow...
3,You’ve always been hyper alert to your surroun...,loss prevent hotel secur call,0,alway hyper alert surround uncanni sens thing ...
4,Your surprise is waiting! Explore career oppor...,loss prevent detect millburi,0,surpris wait explor career opportun marshal st...
...,...,...,...,...
19995,"PetroChoice, a leading distributor of lubrican...",warehous specialist 9a 6 30p,19,petrochoic lead distributor lubric oil fluid g...
19996,Old Dominion is currently recruiting for a Doc...,dockwork part time 4 pm 10 pm,19,old dominion current recruit dock worker role ...
19997,"At American Signature Inc., we believe everyon...",warehous associ,19,american signatur inc believ everyon right wel...
19998,"Pennsylvania 700 Uline Way, Allentown, PA 1810...",warehous associ night 25 per hour,19,pennsylvania 700 ulin way allentown pa 18106 u...


In [14]:
df.to_parquet('/content/drive/MyDrive/jd_data_processed.parquet.snappy', compression='snappy')

In [2]:
df = pd.read_parquet('/content/drive/MyDrive/jd_data_processed.parquet.snappy')
df

,text,title,label,cleantext
0,Search by Keyword More Options Search by Locat...,loss prevent detect colma,0,search keyword option search locat clear load ...
1,Description At more than 80 award-winning prop...,loss prevent offic pm,0,descript 80 award win properti worldwid ritz c...
2,COMPANY DESCRIPTION: Ethos Risk Services is a ...,p special investig unit field investig siu,0,compani descript etho risk servic nation renow...
3,You’ve always been hyper alert to your surroun...,loss prevent hotel secur call,0,alway hyper alert surround uncanni sens thing ...
4,Your surprise is waiting! Explore career oppor...,loss prevent detect millburi,0,surpris wait explor career opportun marshal st...
...,...,...,...,...
19995,"PetroChoice, a leading distributor of lubrican...",warehous specialist 9a 6 30p,19,petrochoic lead distributor lubric oil fluid g...
19996,Old Dominion is currently recruiting for a Doc...,dockwork part time 4 pm 10 pm,19,old dominion current recruit dock worker role ...
19997,"At American Signature Inc., we believe everyon...",warehous associ,19,american signatur inc believ everyon right wel...
19998,"Pennsylvania 700 Uline Way, Allentown, PA 1810...",warehous associ night 25 per hour,19,pennsylvania 700 ulin way allentown pa 18106 u...


In [31]:
X_train, X_test, y_train, y_test = train_test_split(df[['cleantext','title']], df['label'], test_size=0.2, random_state=42)

In [32]:
vectorizer = TfidfVectorizer(min_df=0.2, max_df=0.7, max_features=1000)
X_train_text = vectorizer.fit_transform(X_train['cleantext']).toarray()
X_train_text = pd.DataFrame(X_train_text)
X_test_text = vectorizer.transform(X_test['cleantext']).toarray()
X_test_text = pd.DataFrame(X_test_text)

vectorizer =  TfidfVectorizer(min_df=5, max_df=0.7, max_features=1000)
X_train_title = vectorizer.fit_transform(X_train['title']).toarray()
X_train_title = pd.DataFrame(X_train_title)
X_test_title = vectorizer.transform(X_test['title']).toarray()
X_test_title = pd.DataFrame(X_test_title)

X_train = pd.concat([X_train.reset_index(drop=True), X_train_text, X_train_title], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_text, X_test_title], axis=1)

X_train = X_train.drop(['cleantext','title'], axis=1)
X_test = X_test.drop(['cleantext','title'], axis=1)

display(X_train)
display(X_test)

,0,1,2,3,4,5,6,7,8,9,...,825,826,827,828,829,830,831,832,833,834
0,0.033685,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.151308,0.000000,0.000000,0.208170,0.000000,0.000000,0.000000,0.128610,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.108473,0.000000,0.031879,0.024873,0.031521,0.029529,0.024398,0.000000,0.153936,0.030225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.045008,0.139278,0.039681,0.000000,0.000000,0.073512,0.000000,0.000000,0.038323,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.022621,0.000000,0.000000,0.000000,0.000000,0.000000,0.030528,0.000000,0.000000,0.037820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.151813,0.000000,0.000000,0.052216,0.000000,0.000000,0.051219,0.000000,0.064632,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15996,0.052351,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15997,0.035806,0.000000,0.000000,0.591141,0.000000,0.000000,0.000000,0.060869,0.060975,0.059862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15998,0.047737,0.000000,0.000000,0.065677,0.000000,0.000000,0.000000,0.000000,0.081293,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,825,826,827,828,829,830,831,832,833,834
0,0.053929,0.000000,0.000000,0.000000,0.047013,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.116611,0.180428,0.000000,0.000000,0.000000,0.000000,0.000000,0.099117,0.000000,0.097478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.108473,0.000000,0.031879,0.024873,0.031521,0.029529,0.024398,0.000000,0.153936,0.030225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.157308,0.000000,0.000000,0.000000,0.091423,0.000000,0.000000,0.089140,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.075612,0.000000,0.074764,0.000000,0.057869,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.075912,0.000000,0.000000,0.000000,0.000000,0.000000,0.102446,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3996,0.222634,0.114825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3997,0.110878,0.068623,0.039102,0.000000,0.000000,0.000000,0.029927,0.000000,0.037764,0.037075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3998,0.114087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.194283,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
model = MultinomialNB()
model.fit(X_train, y_train)
preds = model.predict(X_test)

# Printing accuracy score of the model
print("Accuracy: ", accuracy_score(y_test, preds))

# Classificaton report of the model
#Turning Numbers into Label for confusion matrix
print(classification_report(y_test, preds))
print("\n")
print("\n")
print("Confusion Matrix of Classification")
confusion_matrix(y_test, preds)

Accuracy:  0.555
              precision    recall  f1-score   support

           0       0.10      0.17      0.12       173
           1       0.82      0.87      0.85       179
           2       0.89      0.93      0.91       226
           3       0.06      0.05      0.06       203
           4       0.05      0.05      0.05       199
           5       0.02      0.01      0.01       214
           6       0.73      0.77      0.75       211
           7       0.29      0.26      0.27       202
           8       1.00      1.00      1.00       208
           9       0.26      0.28      0.27       204
          10       0.65      0.73      0.69       194
          11       0.20      0.14      0.17       203
          12       0.95      0.91      0.93       204
          13       0.83      0.73      0.78       209
          14       0.65      0.83      0.73       194
          15       0.71      0.68      0.69       193
          16       0.53      0.67      0.59       202
          

array([[ 29,   0,   1,  43,  53,  35,   0,   0,   0,   2,   2,   0,   1,
          0,   1,   1,   4,   0,   1,   0],
       [  0, 156,   1,   0,   0,   0,   8,   0,   0,   0,   5,   0,   1,
          1,   0,   0,   2,   0,   0,   5],
       [  1,   2, 210,   0,   0,   0,   1,   0,   0,   0,   6,   0,   0,
          3,   0,   1,   1,   0,   0,   1],
       [ 87,   3,   1,  10,  62,  23,   2,   0,   0,   1,   7,   0,   0,
          0,   0,   2,   5,   0,   0,   0],
       [ 90,   1,   0,  52,  10,  29,   3,   0,   0,   1,   6,   0,   0,
          0,   2,   1,   4,   0,   0,   0],
       [ 85,   0,   1,  51,  61,   2,   5,   0,   0,   0,   2,   0,   1,
          0,   0,   0,   6,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 163,   0,   0,   0,   0,   0,   1,
          0,   0,  45,   1,   0,   1,   0],
       [  0,   1,   2,   0,   0,   0,   2,  52,   0,   2,   5,   1,   0,
          4,   4,   0,   6, 123,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 208,   0,   0,  

In [34]:
XGBoost = GradientBoostingClassifier(n_estimators=5)
XGBoost.fit(X_train, y_train)
preds = XGBoost.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, preds))

print(classification_report(y_test, preds))
print("\n")
print("\n")
print("Confusion Matrix of Classification")
confusion_matrix(y_test, preds)

Accuracy:  0.57825
              precision    recall  f1-score   support

           0       0.18      0.23      0.21       173
           1       0.89      0.78      0.83       179
           2       0.83      0.89      0.86       226
           3       0.19      0.26      0.22       203
           4       0.15      0.14      0.14       199
           5       0.17      0.06      0.09       214
           6       0.82      0.70      0.76       211
           7       0.36      0.29      0.32       202
           8       1.00      1.00      1.00       208
           9       0.35      0.35      0.35       204
          10       0.36      0.76      0.49       194
          11       0.32      0.25      0.28       203
          12       0.98      0.89      0.94       204
          13       0.84      0.66      0.74       209
          14       0.72      0.84      0.78       194
          15       0.71      0.73      0.72       193
          16       0.55      0.64      0.59       202
        

array([[ 40,   0,   0,  56,  45,  14,   0,   0,   0,   0,  10,   1,   0,
          1,   3,   0,   3,   0,   0,   0],
       [  1, 139,   1,   0,   0,   1,   7,   0,   0,   1,   9,   0,   2,
          6,   0,   1,   3,   0,   8,   0],
       [  0,   1, 202,   1,   1,   0,   1,   0,   0,   0,  19,   0,   0,
          0,   0,   0,   0,   0,   1,   0],
       [ 58,   0,   2,  52,  50,  21,   2,   0,   0,   0,   8,   0,   0,
          0,   1,   1,   4,   0,   4,   0],
       [ 53,   1,   1,  74,  27,  21,   1,   0,   0,   1,  10,   1,   0,
          2,   2,   0,   3,   0,   2,   0],
       [ 47,   0,   1,  82,  50,  13,   2,   0,   0,   0,   9,   1,   0,
          3,   1,   1,   3,   0,   1,   0],
       [  0,   2,   0,   0,   2,   0, 148,   0,   0,   0,   9,   0,   0,
          1,   0,  49,   0,   0,   0,   0],
       [  3,   1,   4,   0,   0,   0,   1,  58,   0,   0,  19,   1,   0,
          3,   3,   0,   7, 102,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 208,   0,   0,  

In [36]:
tf_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(20, activation="softmax")
])

tf_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

tf_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

tf_preds = tf_model.predict(X_test)
tf_preds = np.array([np.argmax(i) for i in tf_preds])

print(classification_report(y_test, preds))
print("\n")
print("\n")
print("Confusion Matrix of Classification")
confusion_matrix(y_test, preds)

Epoch 1/5
500/500 [==============================] - 2s 4ms/step - loss: 1.3300 - accuracy: 0.5639 - val_loss: 0.8504 - val_accuracy: 0.6227
Epoch 2/5
500/500 [==============================] - 2s 4ms/step - loss: 0.7563 - accuracy: 0.6562 - val_loss: 0.8411 - val_accuracy: 0.6332
Epoch 3/5
500/500 [==============================] - 2s 4ms/step - loss: 0.6884 - accuracy: 0.6702 - val_loss: 0.8141 - val_accuracy: 0.6220
Epoch 4/5
500/500 [==============================] - 2s 4ms/step - loss: 0.6502 - accuracy: 0.6873 - val_loss: 0.8326 - val_accuracy: 0.6277
Epoch 5/5
500/500 [==============================] - 2s 4ms/step - loss: 0.6228 - accuracy: 0.6890 - val_loss: 0.8380 - val_accuracy: 0.6212
              precision    recall  f1-score   support

           0       0.18      0.23      0.21       173
           1       0.89      0.78      0.83       179
           2       0.83      0.89      0.86       226
           3       0.19      0.26      0.22       203
           4       0.15 

array([[ 40,   0,   0,  56,  45,  14,   0,   0,   0,   0,  10,   1,   0,
          1,   3,   0,   3,   0,   0,   0],
       [  1, 139,   1,   0,   0,   1,   7,   0,   0,   1,   9,   0,   2,
          6,   0,   1,   3,   0,   8,   0],
       [  0,   1, 202,   1,   1,   0,   1,   0,   0,   0,  19,   0,   0,
          0,   0,   0,   0,   0,   1,   0],
       [ 58,   0,   2,  52,  50,  21,   2,   0,   0,   0,   8,   0,   0,
          0,   1,   1,   4,   0,   4,   0],
       [ 53,   1,   1,  74,  27,  21,   1,   0,   0,   1,  10,   1,   0,
          2,   2,   0,   3,   0,   2,   0],
       [ 47,   0,   1,  82,  50,  13,   2,   0,   0,   0,   9,   1,   0,
          3,   1,   1,   3,   0,   1,   0],
       [  0,   2,   0,   0,   2,   0, 148,   0,   0,   0,   9,   0,   0,
          1,   0,  49,   0,   0,   0,   0],
       [  3,   1,   4,   0,   0,   0,   1,  58,   0,   0,  19,   1,   0,
          3,   3,   0,   7, 102,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 208,   0,   0,  